In [38]:
import numpy as np
import pandas as pd
import emoji
import datetime

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split  # Import for splitting the data
from gensim.models import KeyedVectors

Emoji Dictionary

In [40]:
emoji_dictionary = {
    0: ":red_heart:",  # Love
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":smiling_face_with_heart-eyes:",  # Adoration
    3: ":loudly_crying_face:",  # Sadness
    4: ":fire:",  # Excitement
    5: ":thumbs_up:",  # Approval
    6: ":folded_hands:",  # Gratitude
    7: ":angry_face:",  # Anger
    8: ":sparkles:",  # Happiness
    9: ":weary_face:",  # Exhaustion
    10: ":astonished_face:",  # Surprise
    11: ":confused_face:",  # Confusion
    12: ":tropical_drink:",  # Celebration
    13: ":broken_heart:",  # Heartbreak
    14: ":thinking_face:",  # Contemplation
    15: ":sleeping_face:",  # Sleepiness
    16: ":victory_hand:",  # Success
    17: ":thumbs_down:",  # Disapproval
    18: ":grimacing_face:",  # Discomfort
    19: ":smiling_face_with_halo:",  # Innocence
}


Function to get labels from CLDR names

In [41]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

Reading data

In [42]:
data = pd.read_csv('data/emoji_data/emoji_data.csv', header=None)
data.head()

,0,1
0,When your alarm goes off for the fifth time,9
1,That moment when someone eats the last slice o...,7
2,When you finally finish a project,8
3,Me trying to understand the group chat,11
4,When the music is too loud at the party,18


In [43]:
X = data[0].values
Y = data[1].values


Embeddings

With glove dataset

In [44]:
# with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
#     content = file.readlines()

In [ ]:
# embeddings = {}

# for line in content:
#     line = line.split()
#     embeddings[line[0]] = np.array(line[1:], dtype=float)

With crawl dataset

In [45]:
fasttext_model_path = "data/fast_text/crawl_dataset/crawl-300d-2M-subword.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path)


In [46]:
# Convert FastText embeddings to a dictionary
embeddings = {}
for word in fasttext_model.index_to_key:
    embeddings[word] = fasttext_model.get_vector(word)


Convert input text into tokens

In [47]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index


In [87]:
Xtokens = tokenizer.texts_to_sequences(X)


In [49]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)

maxlen

11

In [50]:
Xtrain = pad_sequences(Xtokens, maxlen=maxlen, padding='post', truncating='post')


In [51]:
Ytrain = to_categorical(Y)

Split data into train and test set

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=42)

Model

In [53]:
embed_size = 300

In [54]:
embedding_matrix = np.zeros((len(word_to_index) + 1, embed_size))

for word, i in word_to_index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector
    else:
        # Handle out-of-vocabulary words or phrases by aggregating subword embeddings
        phrase_embed_sum = None
        for subword in word.split():
            if subword in embeddings:
                if phrase_embed_sum is None:
                    phrase_embed_sum = embeddings[subword]
                else:
                    phrase_embed_sum += embeddings[subword]
        if phrase_embed_sum is not None:
            # Take the average of subword embeddings
            embedding_matrix[i] = phrase_embed_sum / len(word.split())

Code to generate file name for saving model

In [70]:
def returnfilename(prefix="model"):
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"{prefix}{formatted_datetime}epochs:{epochs}_layers:{layer1}_{layer2}_{layer3}_{layer4}"
    return filename


In general more layers >>

In [78]:
epochs = 1000
#units
layer1 = 256
layer2 = 16
layer3 = 4
layer4 = 2

In [79]:
model = Sequential([
    Embedding(input_dim=len(word_to_index) + 1,
              output_dim=embed_size,
              input_length=maxlen,
              weights=[embedding_matrix],
              trainable=False),
    LSTM(units=layer1, return_sequences=True),
    LSTM(units=layer2, return_sequences=True),
    LSTM(units=layer3, return_sequences=True),
    LSTM(units=layer4),
    Dense(20, activation='softmax')  # Set output dimensionality to 20
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
model.fit(Xtrain, Ytrain, epochs=epochs)

Epoch 1/1000


5/5 [==============================] - 6s 19ms/step - loss: 2.9938 - accuracy: 0.0063
Epoch 2/1000
5/5 [==============================] - 0s 21ms/step - loss: 2.9777 - accuracy: 0.0250
Epoch 3/1000
5/5 [==============================] - 0s 22ms/step - loss: 2.9618 - accuracy: 0.1000
Epoch 4/1000
5/5 [==============================] - 0s 20ms/step - loss: 2.9419 - accuracy: 0.0938
Epoch 5/1000
5/5 [==============================] - 0s 19ms/step - loss: 2.9150 - accuracy: 0.1562
Epoch 6/1000
5/5 [==============================] - 0s 19ms/step - loss: 2.8817 - accuracy: 0.2188
Epoch 7/1000
5/5 [==============================] - 0s 22ms/step - loss: 2.8557 - accuracy: 0.2500
Epoch 8/1000
5/5 [==============================] - 0s 19ms/step - loss: 2.8357 - accuracy: 0.2500
Epoch 9/1000
5/5 [==============================] - 0s 18ms/step - loss: 2.8775 - accuracy: 0.1813
Epoch 10/1000
5/5 [==============================] - 0s 15ms/step - loss: 2.8288 - accuracy: 0.2562
Epoch 11/1000
5/5 [===

In [81]:
filename = returnfilename()
model.save(f"models/{filename}.hdf5")

In [88]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

1/1 [==============================] - 0s 39ms/step - loss: 1.4655 - accuracy: 0.4375
Test Loss: 1.4654505252838135
Test Accuracy: 0.4375


In [83]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted and actual labels from one-hot encoded format to integer labels
y_pred_labels = np.argmax(y_pred, axis=1)
Y_test_labels = np.argmax(Y_test, axis=1)

# Compare predicted labels with actual labels
correct_predictions = np.sum(y_pred_labels == Y_test_labels)
total_predictions = len(Y_test_labels)
accuracy = correct_predictions / total_predictions

# Print accuracy
print("Accuracy:", accuracy)

# Display predicted labels and actual labels
for i in range(len(y_pred_labels)):
    if y_pred_labels[i]!=Y_test_labels[i]:
        print("Predicted:", label_to_emoji(y_pred_labels[i]), "Actual:", label_to_emoji(Y_test_labels[i]))


1/1 [==============================] - 2s 2s/step
Accuracy: 0.4375
Predicted: 😕 Actual: ✌️
Predicted: 😩 Actual: 👍
Predicted: 😬 Actual: 😂
Predicted: 😬 Actual: 😴
Predicted: 😕 Actual: 🤔
Predicted: 😩 Actual: 🙏
Predicted: 😭 Actual: 😲
Predicted: 😕 Actual: ✌️
Predicted: 😬 Actual: 😠
Predicted: 😭 Actual: 🍹
Predicted: 😬 Actual: 😠
Predicted: 😬 Actual: 😠
Predicted: 😬 Actual: ✨
Predicted: 😕 Actual: 😩
Predicted: 😕 Actual: ✌️
Predicted: 😕 Actual: 🤔
Predicted: 😕 Actual: 🤔
Predicted: 😩 Actual: 🙏


In [85]:
# Make predictions
test = ["I am trying", "I want to cry", "This is just sad"]
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')
y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 31ms/step


In [86]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am trying 😭
I want to cry 😭
This is just sad 😕
